## COVID-19 Data Visualization by Country

In [1]:
import requests
import ipywidgets
import json
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import datetime

source = "https://pomber.github.io/covid19/timeseries.json"
r = requests.get(source)
data_covid = r.json()

country_list = [country for country in data_covid.keys()]
country_list.sort()

# data_covid  = {country1: [{'date': 'yyyy-m-dd', 'confirmed': ?, 'deaths': ?, 'recovered': ?}, ...], 
#                country2: [{'date': 'yyyy-m-dd', 'confirmed': ?, 'deaths': ?, 'recovered': ?}, ...], ...}

DropDown1 = ipywidgets.Dropdown(options=country_list, description="Countries")   # display(ListBox)
CheckBox1 = ipywidgets.Checkbox(value=True, description="Show confirmed")
CheckBox2 = ipywidgets.Checkbox(value=True, description="Show deaths")
CheckBox3 = ipywidgets.Checkbox(value=True, description="Show recovered")
RadioButton1 = ipywidgets.RadioButtons(options=["Linear", "Logaritmic"], description="Y Scale")

GUI = ipywidgets.HBox([ipywidgets.VBox([DropDown1, RadioButton1]), 
                       ipywidgets.VBox([CheckBox1, CheckBox2, CheckBox3])])

#@ipywidgets.interact(country=DropDown1, 
#                     show_confirmed=CheckBox1,
#                     show_deaths=CheckBox2,
#                     show_recovered=CheckBox3, 
#                     yaxis_type=RadioButton1)
def plot_covid_data(country, show_confirmed, show_deaths, show_recovered, yaxis_type):
    country_data_list = data_covid.get(country)
    date_data = [data_dict.get('date') for data_dict in country_data_list]
    confirmed_data = [data_dict.get('confirmed') for data_dict in country_data_list]
    death_data = [data_dict.get('deaths') for data_dict in country_data_list]
    recovered_data = [data_dict.get('recovered') for data_dict in country_data_list]
    n_elements = len(country_data_list)
    
    text_data = f"Date: {date_data[-1]}:\n * Confirmed: {confirmed_data[-1]:,}\n * Deaths: {death_data[-1]:,}\n * Recovered: {recovered_data[-1]:,}"
    
    data_array = np.concatenate([np.array(confirmed_data).reshape(n_elements, 1), 
                                 np.array(death_data).reshape(n_elements, 1), 
                                 np.array(recovered_data).reshape(n_elements, 1)], axis=1)
    
    dates = [datetime.datetime.strptime(d, "%Y-%m-%d").date() for d in date_data]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    if show_confirmed:
        if yaxis_type == 'Linear':
            ax.plot(dates, data_array[:,0], '-o', label="Confirmed")
        else:
            ax.semilogy(dates, data_array[:,0], '-o', label="Confirmed")
    
    if show_deaths:
        if yaxis_type == 'Linear':
            ax.plot(dates, data_array[:,1], '-s', label="Deaths")
        else:
            ax.semilogy(dates, data_array[:,1], '-s', label="Deaths")
                
    if show_recovered:
        if yaxis_type == 'Linear':
            ax.plot(dates, data_array[:,2], '-^', label="Recovered")
        else:
            ax.semilogy(dates, data_array[:,2], '-^', label="Recovered")
    
    ax.set_title(f"Data registered by Country - {country}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Population")
    ax.legend()
    ax.grid(linestyle='--', which='both')
    
    ax.text(ax.get_xlim()[1] + 3, 3.5*ax.get_ylim()[1]/4, text_data, bbox={'facecolor': 'pink'})
    
    plt.show()
    

out = ipywidgets.interactive_output(plot_covid_data, {'country': DropDown1,
                                                      'show_confirmed': CheckBox1,
                                                      'show_deaths': CheckBox2,
                                                      'show_recovered': CheckBox3, 
                                                      'yaxis_type': RadioButton1})
display(GUI, out)

Output()